# ArcGIS Online Web Map Audit

Description: This will audit all of the webmaps in an organization.
 
Beginning with: AGOWebMapAuditPythonAPI.py
 
Created on: 2/13/2018
 
Purpose: The AGO Web Map Audit loops through all of the web maps in an organization and loops through the layers within the web map and creates an excel sheet that lists all of the map service urls within each web map, as well as, comparing the sharing status between the Webmap and Layers. 
 
Authored by: Joe Guzi
 
Previous Production Date: 9/3/18      Production Date: 12/26/18

# Connect to ArcGIS Online

In [ ]:
# import
from arcgis.gis import GIS
from arcgis.gis.server import Server
import getpass
from IPython.display import display
from arcgis.mapping import WebMap
import json
import requests
import csv
from tkinter.filedialog import asksaveasfilename

# Connection Variables
Organization = input("What's your oranization? ")
User = input("What's your  username? ")
Password = getpass.getpass('Password: ')

# Connection 
gis = GIS(Organization, User, Password, verify_cert=False)
gis
Audit = gis.content.search(query="", item_type="Web Map", max_items = 10000000)

# Audit Section

In [ ]:
# Functions
def CompareSharing(ItemAccess, LayerAccess):
    Issue = "No"
    
    if ItemAccess == "org" and LayerAccess == "private":
        Issue = "Yes"
    elif ItemAccess == "public" and LayerAccess == "private":
        Issue = "Yes"
    elif ItemAccess == "public" and LayerAccess == "org":
        Issue = "Yes"
    elif ItemAccess == "shared":
        Issue = "Maybe"
    elif LayerAccess == "shared":
        Issue = "Maybe"
    return Issue
# End Function Section

s = ""
    
for Item in Audit:
    ItemTitle = str(Item["title"].replace(',', ""))
    ItemID = str(Item["id"])
    ItemType = str(Item["type"])
    ItemOwner = str(Item["owner"])
    ItemAccess = str(Item["access"])
    ItemData = Item.get_data(try_json=True)
    for ItemInfo in ItemData:
        # Layers
        if ItemInfo == "operationalLayers":
            layers = ItemData["operationalLayers"]
            for layer in layers:
                #print (layer)
                s += ItemTitle.replace(",", "") + ","
                s += ItemID + ","
                s += ItemType + ","
                s += ItemOwner + ","
                s += ItemAccess + ","
                s += "Operational Layer" + ","
                s += str(layer.get("id")) + ","
                s += str(layer.get("title")).replace(",", "") + ","
                s += str(layer.get("layerType")) + ","
                s += str(layer.get("visibility")) + ","
                LayerURL = str(layer.get('url'))
                s += LayerURL + ","
                LayerItemID = str(layer.get('itemId'))
                if LayerItemID != "None":
                    s += LayerItemID + ","
                    LayerItem = gis.content.search(query='id:' + str(LayerItemID))
                    if LayerItem != []:
                        LayerAccess = str(LayerItem[0].get("access"))
                        s += LayerAccess + ","
                        s += "Yes" + ","
                        SharingIssue = CompareSharing(ItemAccess, LayerAccess)
                        s += SharingIssue + ","
                    else:
                        s += ","
                        s += "No" + ","
                        s += "Maybe" + ","
                else:
                    s += ","
                s += "\n"
        # Tables
        if ItemInfo == "tables":
            tables = ItemData["tables"]
            for table in tables:
                #print (table)
                s += ItemTitle.replace(",", "") + ","
                s += ItemID + ","
                s += ItemType + ","
                s += ItemOwner + ","
                s += ItemAccess + ","
                s += "Table" + ","
                s += str(table.get("id")) + ","
                s += str(table.get("title")).replace(",", "") + ","
                s += ","
                s += "," 
                s += str(table.get("url")) + "," #"" + ","
                LayerItemID = str(table.get('itemId'))
                if LayerItemID != "None":
                    s += LayerItemID + ","
                    LayerItem = gis.content.search(query='id:' + str(LayerItemID))
                    if LayerItem != []:
                        LayerAccess = str(LayerItem[0].get("access"))
                        s += LayerAccess + ","
                        s += "Yes" + ","
                        SharingIssue = CompareSharing(ItemAccess, LayerAccess)
                        s += SharingIssue + ","
                    else:
                        s += ","
                        s += "No" + ","
                        s += "Maybe" + ","
                else:
                    s += ","
                s += "\n"
        # Basemap
        if ItemInfo == "baseMap":
            basemaps = ItemData["baseMap"]
            BaseMapLayers = basemaps["baseMapLayers"]
            for basemaplayer in BaseMapLayers:
                #print (basemaplayer)
                s += ItemTitle.replace(",", "") + ","
                s += ItemID + ","
                s += ItemType + ","
                s += ItemOwner + ","
                s += ItemAccess + ","
                s += "Basemap" + ","
                s += str(basemaplayer.get("id")) + ","
                s += str(basemaplayer.get("title")).replace(",", "") + ","
                s += str(basemaplayer.get("layerType")) + ","
                s += "," 
                s += str(basemaplayer.get("url")) + ","
                LayerItemID = str(basemaplayer.get('itemId'))
                if LayerItemID != "None":
                    s += LayerItemID + ","
                    LayerItem = gis.content.search(query='id:' + str(LayerItemID))
                    if LayerItem != []:
                        LayerAccess = str(LayerItem[0].get("access"))
                        s += LayerAccess + ","
                        s += "Yes" + ","
                        SharingIssue = CompareSharing(ItemAccess, LayerAccess)
                        s += SharingIssue + ","
                    else:
                        s += ","
                        s += "No" + ","
                        s += "Maybe" + ","
                else:
                    s += ","
                s += "\n"

# Write the Audit Results to Excel

In [ ]:
filename = asksaveasfilename() # show an "Open" dialog box and return the path to the selected file
print(filename)
csvfile = filename + ".csv"
print(csvfile)
with open(csvfile, 'w') as output:
    print ("Wrtie data To CSV File")
    output.write("ItemTitle,ItemId,ItemType,ItemOwner,ItemAccess,MapLocation,LayerID,LayerTitle,LayerType,LayerVisisbility,LayerURL,ItemID,LayerAccess,OrganizationalLayer,SharingIssue\n")
    output.writelines(s)
    print("Wrtie data To CSV File Complete!")